# Widgets

This example uses slider widgets to adjust parameters

In [5]:
import dpsim as dps
import dpsim.components.dp as dp

In [22]:
import math
import ipywidgets

pha = ipywidgets.FloatSlider(description="phase [rad]", min=-math.pi, max=math.pi)
mag = ipywidgets.FloatSlider(description="magnitude [V]", value = 50)
res = ipywidgets.FloatSlider(description="resistance [Ohm]", value = 50)
ind = ipywidgets.FloatSlider(description="Inductance [H]", max = 1)
cap = ipywidgets.FloatSlider(description="capactance [F]", max = 1)

display(pha, mag, res, ind, cap)

FloatSlider(value=0.0, description='phase [rad]', max=3.141592653589793, min=-3.141592653589793)

FloatSlider(value=50.0, description='magnitude [V]')

FloatSlider(value=50.0, description='resistance [Ohm]')

FloatSlider(value=0.0, description='Inductance [H]', max=1.0)

FloatSlider(value=0.0, description='capactance [F]', max=1.0)

In [20]:
import cmath
v = cmath.rect(magnitude.value, phase.value)
r = resistance.value
l = inductance.value
c = capacitance.value

gnd = dps.Node.GND()
n1 = dps.Node("n1")
n2 = dps.Node("n2")

vin = dp.VoltageSource("v_in", [n1, gnd], v)
c1 = dp.Capacitor("c1", [n1, gnd], c)
l1 = dp.Inductor("l1", [n1, gnd], l)
r1 = dp.Resistor("r1", [n1, gnd], r)

In [21]:
sys = dps.SystemTopology(50, [gnd, n1, n2], [vin, c1, l1, r1])
sim = dps.Simulation("Widgets", sys, duration=0.03, timestep=1e-4)

In [23]:
sim.run()

In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib as plt
import dataprocessing.plottools as pt
import dataprocessing.readtools as rt

results     = rt.read_timeseries_dpsim_cmpl('Logs/Widgets_LeftVector.csv')
results_emt = [ series.dynphasor_shift_to_emt('%s_emt' % series.name, 50) for series in results ]

for series in results_emt:
    pt.plot_timeseries('Results EMT', series)

TypeError: complex() can't take second arg if first is a string